In [35]:
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
np.set_printoptions(suppress=True)

In [28]:
def interp_pix_arrays(img_bands, band: int):
    im = Image.fromarray(img_bands[band, :, :])
    im = im.resize((294, 331), resample=3) # bicubic interpolation --> negative values; bilinear interpolation --> no negative values
    pix = np.array(im)
    return pix

In [25]:
# Explore gfs image
loc1_apr_gfs = rio.open('data/ee/weather-aq-datalayer-location1/gfs/waq_location1_apr_gfs.tif')
loc1_apr_o3 = rio.open(folder_path + 'o3/waq_location1_apr_o3.tif')

temp = loc1_apr_gfs.read(1)
density = loc1_apr_o3.read(1)
density.shape

(835, 1164)

In [33]:
def gfs_to_array(path):
    gfs_img = rio.open(path)
    gfs_allbands = gfs_img.read()
    shp = gfs_allbands.shape

    data = []
    for band in range(1, 4): # cutting 21, 22, 23
        pix = interp_pix_arrays(gfs_allbands, band)
        data.append(pix)
        
    data = np.stack(data)    
    print(data.shape)
    
    return data

In [42]:
def s5p_to_array(path):
    s5p_img = rio.open(path)
    s5p_allbands = s5p_img.read()
    
    data = []
    pix = interp_pix_arrays(s5p_allbands, 1)
    data.append(pix)
        
    data = np.stack(data)    
    print(data.shape)
    
    return data

In [39]:
folder_path = 'data/ee/weather-aq-datalayer-location1/'

def waq_location1(dataset):
    waq_data = []
    waq_path = 'data/ee/weather-aq-datalayer-location1/' + dataset + '/'
    for filename in os.listdir(waq_path):
        if filename.endswith(".tif"): 
            path = waq_path + filename
            
            if dataset == 'gfs':
                data = gfs_to_array(path)
            else:
                data = s5p_to_array(path)

            waq_data.append(data)
            continue

        else:
            continue

    print('WAQ data for {} dataset compiled'.format(dataset))
    waq_data_final = np.stack(waq_data)
    
    waq_data_final = np.concatenate(waq_data_final, axis=1)
    waq_data_final = np.nan_to_num(waq_data_final)

    # print(waq_data_final.shape)

    return waq_data_final

In [44]:
gfs_array = waq_location1('gfs')
no2_array = waq_location1('no2')
so2_array = waq_location1('so2')
o3_array = waq_location1('o3')

(3, 331, 294)
(3, 331, 294)
(3, 331, 294)
(3, 331, 294)
WAQ data for gfs dataset compiled
(3, 1324, 294)
(1, 331, 294)
(1, 331, 294)
(1, 331, 294)
(1, 331, 294)
WAQ data for no2 dataset compiled
(1, 1324, 294)
(1, 331, 294)
(1, 331, 294)
(1, 331, 294)
(1, 331, 294)
WAQ data for so2 dataset compiled
(1, 1324, 294)
(1, 331, 294)
(1, 331, 294)
(1, 331, 294)
(1, 331, 294)
WAQ data for o3 dataset compiled
(1, 1324, 294)


In [53]:
aq_array = np.concatenate([no2_array, so2_array, o3_array], axis=0)
waq_array = np.concatenate([gfs_array, aq_array], axis=0)
waq_array[0, 0, 0]

0.007300396

In [56]:
rows, cols = np.mgrid[slice(waq_array.shape[1]), slice(waq_array.shape[2])]

import itertools as it

pix_1d_rows = []
for col in range(0, 294):    
    for row in range(0, 1324):
        out = waq_array[:, row, col]
        # print(out.shape)
        pix_1d_rows.append(out)

len(pix_1d_rows)

389256

In [57]:
waq_2d_final = np.stack(pix_1d_rows, axis=0)
# np.unique(pix_2darray_final, return_counts=True)
# pix_2darray_final.shape
np.save('weather-aq-datalayer-final.npy', waq_2d_final)